In [10]:
import torch
from torch import nn
from lib.config import *
from lib.data.dataloading import load_raw

In [7]:
WINSIZE = 1001
trainloader, testloader = load_raw(
    RAW_DIR,
    WINSIZE,
    n_hours=0.5,
    sessions=['2023-11-02_13_55_22'],
    test_size=0.5,
    batch_size=32,
    shuffle_test=True,
    chunk_len_hrs=0.01
)

Using Directories: ['2023-11-02_13_55_22']
Index: 0, Date: 2023-11-02_13_55_22, nSamples: 2499818, Time Elapsed: 13:04:33.095864, Time Recorded: 6:56:38.180000
Created 694 chunks of length 3600 samples each
Randomly selected 50 chunks
Total train length: 0:19:10 (115000 Samples)
Total test length: 0:19:10 (115000 Samples)


In [52]:
class ResBlockMAE(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, seq_len, relu=True, p_dropout=None):
        super().__init__()
        self.use_relu = relu
        self.c = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.LayerNorm((out_channels, seq_len)),
            nn.ReLU(),
            nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.LayerNorm((out_channels, seq_len)),
            nn.ReLU(),
            nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.LayerNorm((out_channels, seq_len)),
        )
        if self.use_relu:
            self.c.add_module('relu', nn.ReLU())
        if p_dropout is not None:
            self.c.add_module('dropout', nn.Dropout(p=p_dropout))

        self.identity = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.LayerNorm((out_channels, seq_len))
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.c(x) + self.identity(x)
        return self.relu(x) if self.use_relu else x

# pytorch model with transformer encoder
class TransformerEncoder(nn.Module):
    def __init__(self):
        super(TransformerEncoder, self).__init__()
        self.e = nn.Sequential(
            ResBlockMAE(in_channels, enc_dims[0], 9, 'same', winsize),
            *[ResBlockMAE(self.enc_dims[i], self.enc_dims[i+1], 3, 'same', winsize, p_dropout=p_dropout) for i in range(len(self.enc_dims)-1)]
        )
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(3, 1, 2048, 0.1, batch_first=True), 
            1,
            enable_nested_tensor=False
        )

    def forward(self, x):
        x = self.transformer_encoder(x)
        return x

In [53]:
model = TransformerEncoder()
for k,v in model.state_dict().items():
    print(k,': ',v.shape)

transformer_encoder.layers.0.self_attn.in_proj_weight :  torch.Size([9, 3])
transformer_encoder.layers.0.self_attn.in_proj_bias :  torch.Size([9])
transformer_encoder.layers.0.self_attn.out_proj.weight :  torch.Size([3, 3])
transformer_encoder.layers.0.self_attn.out_proj.bias :  torch.Size([3])
transformer_encoder.layers.0.linear1.weight :  torch.Size([2048, 3])
transformer_encoder.layers.0.linear1.bias :  torch.Size([2048])
transformer_encoder.layers.0.linear2.weight :  torch.Size([3, 2048])
transformer_encoder.layers.0.linear2.bias :  torch.Size([3])
transformer_encoder.layers.0.norm1.weight :  torch.Size([3])
transformer_encoder.layers.0.norm1.bias :  torch.Size([3])
transformer_encoder.layers.0.norm2.weight :  torch.Size([3])
transformer_encoder.layers.0.norm2.bias :  torch.Size([3])


In [54]:
t = torch.Tensor(32, 1001, 3)
x = model(t)